In [4]:
# Load in our libraries
import pandas as pd
import numpy as np
import re
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

import warnings
warnings.filterwarnings('ignore')

# Going to use these 5 base models for the stacking
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import KFold;

/home/tapas/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import os
os.chdir('/home/tapas/Downloads/Brainwaves/competition/')

In [73]:
# Load in the train and test datasets
train = pd.read_csv('new_train.csv')[1:6000]
test = pd.read_csv('test.csv')

# Store our passenger ID for easy access
TransactionId = test['transaction_id']

train.head(3)

,transaction_id,num_var_1,num_var_2,num_var_3,num_var_4,num_var_5,num_var_6,num_var_7,cat_var_1,cat_var_2,...,cat_var_34,cat_var_35,cat_var_36,cat_var_37,cat_var_38,cat_var_39,cat_var_40,cat_var_41,cat_var_42,target
1,id_657348,8.421053e-08,0.031934,0.0,0.000003,3.361842e-07,4.407895e-08,6.117695e-08,to,ce,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,id_585898,4.901316e-07,0.153431,0.0,0.000002,2.769737e-07,3.026316e-07,2.596834e-07,da,tn,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,id_443360,8.421053e-08,0.663424,0.0,0.000666,1.312638e-04,3.269738e-03,8.302901e-04,ep,ep,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [74]:
drop_column = ['transaction_id']
train.drop(drop_column, axis=1, inplace = True)
test.drop(drop_column, axis=1, inplace = True)

In [75]:
train = train.dropna(subset = ['target'])

In [76]:
train.head(5)

,num_var_1,num_var_2,num_var_3,num_var_4,num_var_5,num_var_6,num_var_7,cat_var_1,cat_var_2,cat_var_3,...,cat_var_34,cat_var_35,cat_var_36,cat_var_37,cat_var_38,cat_var_39,cat_var_40,cat_var_41,cat_var_42,target
1,8.421053e-08,0.031934,0.0,0.000003,3.361842e-07,4.407895e-08,6.117695e-08,to,ce,tn,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,4.901316e-07,0.153431,0.0,0.000002,2.769737e-07,3.026316e-07,2.596834e-07,da,tn,yq,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,8.421053e-08,0.663424,0.0,0.000666,1.312638e-04,3.269738e-03,8.302901e-04,ep,ep,mg,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.834803e-05,0.159882,0.0,0.000002,2.769737e-07,1.372237e-05,1.258047e-05,da,tn,ax,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.147809e-04,0.185577,0.0,0.000009,1.124342e-06,1.276493e-04,1.107325e-04,jy,ce,tj,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [77]:
na_totals = train.isnull().sum().sort_values(ascending=False)
na_totals[na_totals>0]

cat_var_8    1583
cat_var_3     668
cat_var_1     198
dtype: int64

In [78]:
full_data = [train, test]

In [79]:
for dataset in full_data:
    dataset['num_var_1'] = np.log1p(dataset['num_var_1'])
    dataset['num_var_2'] = np.log1p(dataset['num_var_2'])
    dataset['num_var_3'] = np.log1p(dataset['num_var_3'])
    dataset['num_var_4'] = np.log1p(dataset['num_var_4'])
    dataset['num_var_5'] = np.log1p(dataset['num_var_5'])
    dataset['num_var_6'] = np.log1p(dataset['num_var_6'])
    dataset['num_var_7'] = np.log1p(dataset['num_var_7'])

In [80]:
###COMPLETING: complete or delete  missing values in train and test/validation dataset
for dataset in full_data:    
    #complete embarked with mode
    dataset['cat_var_3'].fillna(dataset['cat_var_3'].mode()[0], inplace = True)
    dataset['cat_var_6'].fillna(dataset['cat_var_6'].mode()[0], inplace = True)
    dataset['cat_var_1'].fillna(dataset['cat_var_1'].mode()[0], inplace = True)
    dataset['cat_var_8'].fillna(dataset['cat_var_8'].mode()[0], inplace = True)

In [81]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [82]:
#CONVERT: convert objects to category using Label Encoder for train and test/validation dataset
label = LabelEncoder()

for dataset in full_data: 
    dataset['cat_var_1'] = label.fit_transform(dataset['cat_var_1'])
    dataset['cat_var_2'] = label.fit_transform(dataset['cat_var_2'])
    dataset['cat_var_3'] = label.fit_transform(dataset['cat_var_3'])
    dataset['cat_var_4'] = label.fit_transform(dataset['cat_var_4'])
    dataset['cat_var_5'] = label.fit_transform(dataset['cat_var_5'])
    dataset['cat_var_6'] = label.fit_transform(dataset['cat_var_6'])
    dataset['cat_var_7'] = label.fit_transform(dataset['cat_var_7'])
    dataset['cat_var_8'] = label.fit_transform(dataset['cat_var_8'])
    dataset['cat_var_9'] = label.fit_transform(dataset['cat_var_9'])
    dataset['cat_var_10'] = label.fit_transform(dataset['cat_var_10'])
    dataset['cat_var_11'] = label.fit_transform(dataset['cat_var_11'])
    dataset['cat_var_12'] = label.fit_transform(dataset['cat_var_12'])
    dataset['cat_var_13'] = label.fit_transform(dataset['cat_var_13'])
    dataset['cat_var_14'] = label.fit_transform(dataset['cat_var_14'])
    dataset['cat_var_15'] = label.fit_transform(dataset['cat_var_15'])
    dataset['cat_var_16'] = label.fit_transform(dataset['cat_var_16'])
    dataset['cat_var_17'] = label.fit_transform(dataset['cat_var_17'])
    dataset['cat_var_18'] = label.fit_transform(dataset['cat_var_18'])

In [83]:
train.head(5)

,num_var_1,num_var_2,num_var_3,num_var_4,num_var_5,num_var_6,num_var_7,cat_var_1,cat_var_2,cat_var_3,...,cat_var_34,cat_var_35,cat_var_36,cat_var_37,cat_var_38,cat_var_39,cat_var_40,cat_var_41,cat_var_42,target
1,8.421052e-08,0.031435,0.0,0.000003,3.361842e-07,4.407895e-08,6.117695e-08,234,1,347,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,4.901315e-07,0.142741,0.0,0.000002,2.769736e-07,3.026315e-07,2.596834e-07,33,18,429,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,8.421052e-08,0.508878,0.0,0.000666,1.312552e-04,3.264404e-03,8.299456e-04,49,4,212,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.834786e-05,0.148318,0.0,0.000002,2.769736e-07,1.372227e-05,1.258039e-05,33,18,17,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.147743e-04,0.170229,0.0,0.000009,1.124341e-06,1.276412e-04,1.107264e-04,118,1,344,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [84]:
test.head(5)

,num_var_1,num_var_2,num_var_3,num_var_4,num_var_5,num_var_6,num_var_7,cat_var_1,cat_var_2,cat_var_3,...,cat_var_33,cat_var_34,cat_var_35,cat_var_36,cat_var_37,cat_var_38,cat_var_39,cat_var_40,cat_var_41,cat_var_42
0,4.736841e-07,0.150776,0.0,2.104998e-06,2.769736e-07,4.828946e-07,1.300010e-07,65,50,157,...,0,0,0,0,0,0,0,0,0,0
1,6.578947e-08,0.082861,0.0,7.499997e-07,1.315789e-07,9.868421e-08,3.186300e-08,322,3,397,...,0,0,0,0,0,0,0,0,0,0
2,1.585526e-07,0.148409,0.0,2.104998e-06,2.769736e-07,1.585526e-07,4.715723e-08,65,50,403,...,0,0,0,0,0,0,0,0,0,0
3,8.552631e-08,0.084417,0.0,7.499997e-07,8.552631e-08,9.868421e-08,4.142190e-08,322,3,116,...,0,0,0,0,0,0,0,0,0,0
4,1.144737e-07,0.085777,0.0,3.549999e-07,4.671053e-08,4.407895e-08,1.452953e-07,126,3,397,...,0,0,0,0,0,0,0,0,0,0


In [85]:
# Some useful parameters which will come in handy later on
ntrain = train.shape[0]
ntest = test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 5 # set folds for out-of-fold prediction
kf = KFold(ntrain, n_folds= NFOLDS, random_state=SEED)

In [86]:
# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=0, params=None):
        params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        print(self.clf.fit(x,y).feature_importances_)
    
# Class to extend XGboost classifer

In [87]:
def get_oof(clf, x_train, y_train, x_test):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(x_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

In [88]:
# Put in our parameters for said classifiers
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 1
}

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':500,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 1
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 1
}

# Support Vector Classifier parameters 
svc_params = {
    'kernel' : 'linear',
    'C' : 0.025
    }

In [89]:
# Create 5 objects that represent our 4 models
rf = SklearnHelper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
ada = SklearnHelper(clf=AdaBoostClassifier, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingClassifier, seed=SEED, params=gb_params)
svc = SklearnHelper(clf=SVC, seed=SEED, params=svc_params)

In [90]:
# Create Numpy arrays of train, test and target ( Survived) dataframes to feed into our models
y_train = train['target'].ravel()
train = train.drop(['target'], axis=1)
x_train = train.values # Creates an array of the train data
x_test = test.values # Creates an array of the test data

In [91]:
# Create our OOF train and test predictions. These base results will be used as new features
et_oof_train, et_oof_test = get_oof(et, x_train, y_train, x_test) # Extra Trees
rf_oof_train, rf_oof_test = get_oof(rf,x_train, y_train, x_test) # Random Forest
ada_oof_train, ada_oof_test = get_oof(ada, x_train, y_train, x_test) # AdaBoost 
gb_oof_train, gb_oof_test = get_oof(gb,x_train, y_train, x_test) # Gradient Boost
svc_oof_train, svc_oof_test = get_oof(svc,x_train, y_train, x_test) # Support Vector Classifier

print("Training is complete")

[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    1.8s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:    0.4s finished
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    2.4s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   10.3s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   23.9s
[Parallel(n_jobs=4)]: Done 500 out of 500 | elapsed:   26.8s finished
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapse

      Iter       Train Loss   Remaining Time 
         1           1.3407           22.32s
         2           1.3087           24.74s
         3           1.2820           24.65s
         4           1.2596           23.20s
         5           1.2403           23.32s
         6           1.2230           23.01s
         7           1.2080           22.59s
         8           1.1917           22.10s
         9           1.1761           21.94s
        10           1.1643           21.70s
        20           1.0844           20.97s
        30           1.0340           20.20s
        40           1.0001           21.21s
        50           0.9799           21.28s
        60           0.9568           20.69s
        70           0.9386           19.53s
        80           0.9219           19.11s
        90           0.9041           18.96s
       100           0.8872           18.77s
       200           0.7661           13.93s
       300           0.6697            9.23s
       40

In [92]:
rf_feature = rf.feature_importances(x_train,y_train)
et_feature = et.feature_importances(x_train, y_train)
ada_feature = ada.feature_importances(x_train, y_train)
gb_feature = gb.feature_importances(x_train,y_train)

[  6.73448946e-02   3.18389259e-02   0.00000000e+00   8.79988632e-02
   4.85032681e-02   5.94305514e-02   1.52425432e-01   1.18249590e-01
   2.20294707e-02   1.74331341e-02   1.39648712e-02   6.25048702e-03
   2.63734205e-02   4.40244433e-04   8.62637106e-02   8.69667631e-03
   1.02574225e-02   5.87461790e-03   2.30779907e-02   4.99615349e-02
   5.14096182e-02   1.12923679e-02   6.13276217e-03   3.67433910e-02
   2.68914618e-02   3.99718830e-03   5.44558365e-03   1.01005493e-02
   1.01374907e-02   6.92878038e-04   5.37794235e-04   0.00000000e+00
   1.95462355e-04   0.00000000e+00   0.00000000e+00   0.00000000e+00
   8.34638583e-06   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00]


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    2.2s finished


[  1.16523585e-02   1.98685304e-02   0.00000000e+00   5.91083694e-03
   6.79141947e-03   7.92637165e-03   2.38769511e-02   9.03846044e-02
   5.47473305e-02   1.12732737e-02   6.32139418e-02   2.93225001e-02
   3.00547242e-02   1.41512513e-03   8.11170333e-02   1.43808026e-02
   8.79497273e-03   8.27378083e-03   3.17380717e-02   4.73134447e-02
   8.86325347e-02   4.64391025e-02   1.91273271e-02   1.07151957e-01
   9.86449100e-02   1.74784976e-02   1.99971888e-02   2.41353890e-02
   2.25360836e-02   1.89741844e-03   4.26579523e-03   2.42675545e-05
   1.43007797e-03   0.00000000e+00   0.00000000e+00   8.26193716e-05
   1.00757777e-04   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00]
[ 0.066  0.188  0.     0.078  0.046  0.064  0.068  0.112  0.006  0.096  0.
  0.014  0.046  0.01   0.07   0.012  0.012  0.004  0.012  0.04   0.012  0.
  0

In [93]:
rf_features = [  6.73448946e-02,   3.18389259e-02,   0.00000000e+00,   8.79988632e-02,
                 4.85032681e-02,   5.94305514e-02,   1.52425432e-01,   1.18249590e-01,
                 2.20294707e-02,   1.74331341e-02,   1.39648712e-02,   6.25048702e-03,
                 2.63734205e-02,   4.40244433e-04,   8.62637106e-02,   8.69667631e-03,
                 1.02574225e-02,   5.87461790e-03,   2.30779907e-02,   4.99615349e-02,
                 5.14096182e-02,   1.12923679e-02,   6.13276217e-03,   3.67433910e-02,
                 2.68914618e-02,   3.99718830e-03,   5.44558365e-03,   1.01005493e-02,
                 1.01374907e-02,   6.92878038e-04,   5.37794235e-04,   0.00000000e+00,
                 1.95462355e-04,   0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
                 8.34638583e-06,   0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
                 0.00000000e+00,   0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
                 0.00000000e+00,   0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
                 0.00000000e+00]
et_features = [ 1.16523585e-02,   1.98685304e-02,   0.00000000e+00,   5.91083694e-03,
                6.79141947e-03,   7.92637165e-03,   2.38769511e-02,   9.03846044e-02,
                5.47473305e-02,   1.12732737e-02,   6.32139418e-02,   2.93225001e-02,
                3.00547242e-02,   1.41512513e-03,   8.11170333e-02,   1.43808026e-02,
                8.79497273e-03,   8.27378083e-03,   3.17380717e-02,   4.73134447e-02,
                8.86325347e-02,   4.64391025e-02,   1.91273271e-02,   1.07151957e-01,
                9.86449100e-02,   1.74784976e-02,   1.99971888e-02,   2.41353890e-02,
                2.25360836e-02,   1.89741844e-03,   4.26579523e-03,   2.42675545e-05,
                1.43007797e-03,   0.00000000e+00,   0.00000000e+00,   8.26193716e-05,
                1.00757777e-04,   0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
                0.00000000e+00,   0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
                0.00000000e+00,   0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
                0.00000000e+00]
ada_features = [ 0.066,  0.188,  0.,     0.078,  0.046,  0.064,  0.068,  0.112,  0.006,  0.096,  0.,
                 0.014,  0.046,  0.01,   0.07,   0.012,  0.012,  0.004,  0.012,  0.04,   0.012,  0.,
                 0.,     0.004,  0.02,   0.008,  0.006,  0.002,  0.,     0.002,  0.002,  0.,     0.,
                 0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
                 0.,     0.,     0.,     0.,     0.,   ]
gb_features = [  0.0620496,   0.18471985,  0.,          0.02798719,  0.01538484,  0.04498921,
                 0.08055583,  0.03744508,  0.00847452,  0.21250444,  0.00144628,  0.01262945,
                 0.05748113,  0.00396104,  0.01450879,  0.05057429,  0.07481115,  0.0268224,
                 0.01049048,  0.02509527,  0.01196303,  0.00112466,  0.00377305,  0.01103084,
                 0.00574739,  0.00267275,  0.00248277,  0.00262637,  0.00388801,  0.00132378,
                 0.0011194,   0.,          0.00031712,  0.,          0.,          0.,          0.,
                 0.,          0.,          0.,          0.,          0.,          0.,          0.,
                 0.,          0.,          0.,          0.,          0. ]

In [94]:
cols = train.columns.values
# Create a dataframe with features
feature_dataframe = pd.DataFrame( {'features': cols,
     'Random Forest feature importances': rf_features,
     'Extra Trees  feature importances': et_features,
      'AdaBoost feature importances': ada_features,
    'Gradient Boost feature importances': gb_features
    })


In [95]:
# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['Random Forest feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['Random Forest feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Random Forest Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['Extra Trees  feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['Extra Trees  feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Extra Trees Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['AdaBoost feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['AdaBoost feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'AdaBoost Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

# Scatter plot 
trace = go.Scatter(
    y = feature_dataframe['Gradient Boost feature importances'].values,
    x = feature_dataframe['features'].values,
    mode='markers',
    marker=dict(
        sizemode = 'diameter',
        sizeref = 1,
        size = 25,
#       size= feature_dataframe['AdaBoost feature importances'].values,
        #color = np.random.randn(500), #set color equal to a variable
        color = feature_dataframe['Gradient Boost feature importances'].values,
        colorscale='Portland',
        showscale=True
    ),
    text = feature_dataframe['features'].values
)
data = [trace]

layout= go.Layout(
    autosize= True,
    title= 'Gradient Boosting Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig,filename='scatter2010')

In [96]:
# Create the new column containing the average of values

feature_dataframe['mean'] = feature_dataframe.mean(axis= 1) # axis = 1 computes the mean row-wise
feature_dataframe.head(3)

,AdaBoost feature importances,Extra Trees feature importances,Gradient Boost feature importances,Random Forest feature importances,features,mean
0,0.066,0.011652,0.06205,0.067345,num_var_1,0.051762
1,0.188,0.019869,0.18472,0.031839,num_var_2,0.106107
2,0.000,0.000000,0.00000,0.000000,num_var_3,0.000000


In [97]:
y = feature_dataframe['mean'].values
x = feature_dataframe['features'].values
data = [go.Bar(
            x= x,
             y= y,
            width = 0.5,
            marker=dict(
               color = feature_dataframe['mean'].values,
            colorscale='Portland',
            showscale=True,
            reversescale = False
            ),
            opacity=0.6
        )]

layout= go.Layout(
    autosize= True,
    title= 'Barplots of Mean Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='bar-direct-labels')

In [98]:
base_predictions_train = pd.DataFrame( {'RandomForest': rf_oof_train.ravel(),
     'ExtraTrees': et_oof_train.ravel(),
     'AdaBoost': ada_oof_train.ravel(),
      'GradientBoost': gb_oof_train.ravel()
    })
base_predictions_train.head()

,AdaBoost,ExtraTrees,GradientBoost,RandomForest
0,1.0,0.0,0.0,0.0
1,1.0,1.0,1.0,0.0
2,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0


In [99]:
data = [
    go.Heatmap(
        z= base_predictions_train.astype(float).corr().values ,
        x=base_predictions_train.columns.values,
        y= base_predictions_train.columns.values,
          colorscale='Viridis',
            showscale=True,
            reversescale = True
    )
]
py.iplot(data, filename='labelled-heatmap')

In [100]:
x_train = np.concatenate(( et_oof_train, rf_oof_train, ada_oof_train, gb_oof_train, svc_oof_train), axis=1)
x_test = np.concatenate(( et_oof_test, rf_oof_test, ada_oof_test, gb_oof_test, svc_oof_test), axis=1)

In [101]:
gbm = xgb.XGBClassifier(
    #learning_rate = 0.02,
 n_estimators= 2000,
 max_depth= 10,
 min_child_weight= 2,
 #gamma=1,
 gamma=0.9,                        
 subsample=0.7,
 colsample_bytree=0.7,
 objective= 'binary:logistic',
 nthread= -1,
 scale_pos_weight=1).fit(x_train, y_train)
predictions = gbm.predict_proba(x_test)

In [102]:
predictions = pd.DataFrame(predictions)

In [103]:
# Generate Submission File 
StackingSubmission = pd.DataFrame({ 'transaction_id': TransactionId,
                            'target': predictions[1] })

In [104]:
StackingSubmission.to_csv("StackingSubmission_3.csv", index=False)